# cml1 - Immobilienrechner
# Data Wrangling (Kaggle Datensatz)
In diesem Notebook konzentrieren wir uns darauf, Rohdaten in eine saubere, analysierbare Form zu bringen. Data Wrangling, auch bekannt als Data Munging, ist ein entscheidender Schritt in der Datenanalyse, um sicherzustellen, dass die Daten korrekt und effizient interpretiert und analysiert werden können.

## Bibliotheken importieren

In [248]:
import pandas as pd
import numpy as np
import re

## Daten laden

In [249]:
df = pd.read_csv('../../data/test_data-Kaggle-v0.11.csv', low_memory=False)
print(df.shape)

(24556, 132)


In [250]:
# print all columns, so we can see all columns
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0.1,Unnamed: 0,Municipality,Living space,Plot area,Floor space,Availability,location,description,detailed_description,url,table,Floor,detail_responsive#municipality,detail_responsive#surface_living,detail_responsive#floor,detail_responsive#available_from,Gemeinde,Wohnfläche,Stockwerk,Nutzfläche,Verfügbarkeit,Grundstücksfläche,detail_responsive#surface_property,detail_responsive#surface_usable,Commune,Surface habitable,Surface du terrain,Surface utile,Disponibilité,Étage,Comune,Superficie abitabile,Disponibilità,Gross return,Piano,Superficie del terreno,Superficie utile,Municipality_merged,Floor_merged,Living_space_merged,Floor_space_merged,Plot_area_merged,Availability_merged,location_parsed,title,details,address,link,details_structured,lat,lon,index,ForestDensityL,ForestDensityM,ForestDensityS,Latitude,Locality,Longitude,NoisePollutionRailwayL,NoisePollutionRailwayM,NoisePollutionRailwayS,NoisePollutionRoadL,NoisePollutionRoadM,NoisePollutionRoadS,PopulationDensityL,PopulationDensityM,PopulationDensityS,RiversAndLakesL,RiversAndLakesM,RiversAndLakesS,WorkplaceDensityL,WorkplaceDensityM,WorkplaceDensityS,Zip,distanceToTrainStation,gde_area_agriculture_percentage,gde_area_forest_percentage,gde_area_nonproductive_percentage,gde_area_settlement_percentage,gde_average_house_hold,gde_empty_apartments,gde_foreigners_percentage,gde_new_homes_per_1000,gde_politics_bdp,gde_politics_cvp,gde_politics_evp,gde_politics_fdp,gde_politics_glp,gde_politics_gps,gde_politics_pda,gde_politics_rights,gde_politics_sp,gde_politics_svp,gde_pop_per_km2,gde_population,gde_private_apartments,gde_social_help_quota,gde_tax,gde_workers_sector1,gde_workers_sector2,gde_workers_sector3,gde_workers_total,Space extracted,rooms,plz_parsed,type_unified,Floor_unified,Plot_area_unified,Living_area_unified,provider,space,address_s,No. of rooms:,Number of apartments:,Surface living:,Land area:,Room height:,Last refurbishment:,Year built:,features,description_detailed,Floor space:,Number of floors:,Volume:,plz,Number of toilets:,Gross yield:,Minimum floor space:,space_cleaned,Type:,Hall height:
0,0,0,Suhr,220 m²,733 m²,NaN,On request,"Galeggenweg 95034 Suhr, AG","6.5 rooms, 220 m²«Landhausvilla mit einzigarti...",DescriptionVilla lädt zum Träumen ein Wir ver...,https://www.immoscout24.ch//en/d/villa-buy-suh...,b <article class=####Box-cYFBPY hKrxoH####><h2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suhr,NaN,220 m²,NaN,733 m²,On request,Strasse: Galeggenweg 9 plz:5034 Stadt: Suhr K...,Landhausvilla mit einzigartigem Umschwung,"6.5 rooms, 220 m²,","Galeggenweg 9, 5034 Suhr, AG",/en/d/villa-buy-suhr/7416568,"{'Municipality': 'Suhr', 'Living space': '220 ...",47.372371,8.075742,19014,0.164382,0.100030,0.063548,47.372110,Suhr,8.075858,0.003811,0.0,0.0,0.272861,0.220974,0.174300,0.240444,0.293830,0.205251,0.020765,0.034714,0.051031,0.097870,0.103867,0.038822,5034,0.487037,27.803959,44.392083,1.131008,26.672950,2.32,0.793301,32.312312,6.361066,4.330690,5.584311,6.795194,16.278076,5.885845,7.346028,NaN,1.445938,20.528040,30.079776,940.677966,9990.0,4212.0,4.281099,6.35,38.0,1015.0,4188.0,5241.0,220.0,5.0,5034,villa,NaN,733.0,220.0,Immoscout24.ch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5034.0,NaN,NaN,NaN,220.0,NaN,NaN
1,1,1,Suhr,230 m²,702 m²,NaN,On request,"Hofstattmattenweg 195034 Suhr, AG","7.5 rooms, 230 m²«Grosszügiges Anwesen mit tra...","DescriptionLicht, Glas und Holz - ein freisteh...",https://www.immoscout24.ch//en/d/detached-hous...,b <article class=####Box-cYFBPY hKrxoH####><h2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Suhr,NaN,230 m²,NaN,702 m²,On request,Strasse: Hofstattmattenweg 19 plz:5034 Stadt:...,Grosszügiges Anwesen mit traumhafter Gartenanlage,"7.5 rooms, 230 m²,","Hofstattmattenweg 19, 5034 Suhr, AG",/en/d/detached-house-buy-suhr/7412921,"{'Municipality': 'Suhr', 'Living space': '230 ...",47.371280,8.0

## Konvertieren von Datentypen

Hier gehen wir die Spalte "details_structured" durch. Wir prüfen, ob es Werte in dieser Spalte gibt, die nicht im Dataframe vorhanden sind. Diese Überprüfung gelingt uns, wenn wir die Spalte "details_structured" in ein Dictionary konvertieren und die Schlüssel des Dictionaries mit den Spaltennamen des Dataframes übereinstimmen. Anschließend wird überprüft, ob der Wert bereits in der Spalte vorhanden ist. Wenn nicht, wird er hinzugefügt.

In [251]:
import ast

def convert_to_dict(val):
    if isinstance(val, str):
        try:
            return ast.literal_eval(val)
        except:
            return {}
    elif isinstance(val, dict):
        return val
    else:
        return {}

df['details_structured'] = df['details_structured'].apply(convert_to_dict)

In [252]:
def update_row_values(row):
    details = row['details_structured']
    for key, value in details.items():
        if key in df.columns and pd.isna(row[key]):
            row[key] = value
    return row

df = df.apply(update_row_values, axis=1)

## Ähnliche Spalten

Wir haben festgestellt, dass es zahlreiche Spalten gibt, die äquivalente Informationen tragen, allerdings in verschiedenen Sprachen oder unter verschiedenen Bezeichnungen. Um keine wertvollen Daten zu verlieren, beabsichtigen wir, diese äquivalenten Spalten in diesem Abschnitt zu vereinen. Zunächst müssen wir jedoch ermitteln, welche dieser Spalten die meisten Informationen enthält, um als Hauptspalte für das Zusammenführen zu dienen. Dies erreichen wir durch das Zählen der fehlenden Werte in jeder Spalte.

### Wohnfläche Spalten

In [253]:
lst = ['Space extracted', 'Living space', 'detail_responsive#surface_living', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'Living_space_merged', 'Surface living:', 'Living_area_unified', 'space_cleaned']
for i in lst:
    print(i, "-", df[i].isna().sum())

Space extracted - 12993
Living space - 13528
detail_responsive#surface_living - 24248
Wohnfläche - 24391
Surface habitable - 24505
Superficie abitabile - 24548
Living_space_merged - 13304
Surface living: - 13377
Living_area_unified - 2125
space_cleaned - 11518


Wir sehen, `Living_area_unified` hat bei weitem die kleinste Anzahl NA-Werte →  Wir werden also 'Living_area_unified' als Hauptspalte mit allen anderen Wochnfläche-Spalten mergen.

### Verfügbarkeit Spalten

In [254]:
lst = ['Availability', 'detail_responsive#available_from', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità', 'Availability_merged']

for i in lst:
    print(i, "-", df[i].isna().sum())

Availability - 12659
detail_responsive#available_from - 24223
Verfügbarkeit - 24384
Disponibilité - 24504
Disponibilità - 24548
Availability_merged - 12427


→  Wir werden `Availability_merged` als Hauptspalte mit allen anderen Verfügbarkeit-Spalten mergen.

Was hier wichtig zu beachten ist, ist dass die Spalte kategorisch ist und viele verschiedene Verfügbarkeits-Daten enthält. Wir werden diese Spalte in den nächsten Schritten weiter bereinigen.

In [255]:
df['Availability_merged'].unique()[:10]

array(['On request', 'Immediately', '01.01.2023', '31.03.2024',
       '01.04.2023', '01.04.2024', '01.12.2024', '31.12.2022',
       '01.09.2023', '01.02.2023'], dtype=object)

### Stockwerk Spalten

In [256]:
lst = ['Floor', 'Floor_unified', 'detail_responsive#floor', 'Étage', 'Piano','Stockwerk', 'Floor_merged']

for i in lst:
    print(i, "-", df[i].isna().sum())

Floor - 14187
Floor_unified - 14187
detail_responsive#floor - 24402
Étage - 24539
Piano - 24553
Stockwerk - 24480
Floor_merged - 19327


→  Wir werden `Floor_unified` als Hauptspalte mit allen anderen Stockwerk-Spalten mergen, da diese Spalte bereits gesäubert ist und keine unnötigen Informationen und Zeichen enthält.

### Nutzfläche Spalten

In [257]:
lst = ['Floor space', 'Nutzfläche', 'detail_responsive#surface_usable', 'Surface utile', 'Superficie utile', 'Floor_space_merged']
for i in lst:
    print(i, "-", df[i].isna().sum())

Floor space - 22248
Nutzfläche - 24515
detail_responsive#surface_usable - 24490
Surface utile - 24549
Superficie utile - 24553
Floor_space_merged - 22197


→  Wir werden `Floor_space_merged` als Hauptspalte mit allen anderen Nutzfläche-Spalten mergen.

### Grundstücksfläche Spalten

In [258]:
lst = ['Plot area', 'detail_responsive#surface_property', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'Plot_area_merged', 'Plot_area_unified', 'Land area:']
for i in lst:
    print(i, "-", df[i].isna().sum())

Plot area - 20286
detail_responsive#surface_property - 24436
Grundstücksfläche - 24497
Surface du terrain - 24529
Superficie del terreno - 24554
Plot_area_merged - 20198
Plot_area_unified - 15280
Land area: - 19550


→  Wir werden `Plot_area_unified` als Hauptspalte mit allen anderen Grundstückfläche-Spalten mergen.

### Postleitzahl Spalten

In [259]:
lst = ['Zip', 'plz', 'plz_parsed']
for i in lst:
    print(i, "-", df[i].isna().sum())

Zip - 0
plz - 0
plz_parsed - 0


→ Hauptspalte: `Zip`. Hier ist ebenfalls kein Merge nötig.

### Zimmer Spalten

- rooms
- No. of rooms:
- details

Uns ist hier aufgefallen, dass die Werte für die Anzahl Zimmer in der Spalten `rooms` und `description` meistens abweichen. Wir werden nun eine neue Spalte `Rooms_new` erstellen, die die Werte aus der Spalte `description` übernimmt, damit wir die beiden Spalten besser vergleichen können.

In [260]:
def extract_rooms(description):
    if isinstance(description, str):
        match = re.search(r'(\d+(\.\d)?)\s*(rooms|Zimmer|Zimmerwohnung)', description)
        if match:
            return float(match.group(1))
    else:
        return np.nan

# Use `assign` method to create the new column
df = df.assign(Rooms_new=df['description'].apply(extract_rooms))

df[['description', 'Rooms_new']]

,description,Rooms_new
0,"6.5 rooms, 220 m²«Landhausvilla mit einzigarti...",6.5
1,"7.5 rooms, 230 m²«Grosszügiges Anwesen mit tra...",7.5
2,"4.5 rooms, 131 m²«Terrassenhaus mit traumhafte...",4.5
3,"6.5 rooms, 140 m²«Mittelhaus in Suhr an attrak...",6.5
4,"4.5 rooms, 156 m²«Bezugsbereit - ruhige Lage u...",4.5
...,...,...
24551,1.5 Zimmerwohnung an zentraler Lage mit schöne...,1.5
24552,OBJEKT IST BEREITS RESERVIERT!!Sie suchen eine...,NaN
24553,Mitten im Ober - Toggenburg ist ein 4 Zimmer C...,4.0
24554,Zu verkaufen ist hier in Wildhaus eine schön u...,5.5


Wenn wir nun die beiden Spalten vergleichen, sehen wir, dass die Werte nicht übereinstimmen.

In [261]:
lst = ['Rooms_new', 'rooms', 'No. of rooms:']
for i in lst:
    print(i, "-", df[i].isna().sum())

Rooms_new - 11096
rooms - 999
No. of rooms: - 13050


`Rooms_new` hat zwar viel mehr fehlende Werte, aber wir gehen davon aus, dass die Spalte `description` mit den Anzahl Zimmer genauer ist als die Spalte `rooms`, da dies die Beschreibung der Immobilie ist. Von wo die Werte aus der Spalte `rooms` stammen, ist nicht klar.

→ Deshalb nehmen wir die neu erstellte Spalte `Rooms_new` als Hauptspalte und mergen bei den NA-Werten mit den Spalten `rooms`

### Immo Typ


In [262]:
df['type_unified'].value_counts()

type_unified
flat                              11046
detached-house                     7164
villa                              1160
semi-detached-house                1043
duplex-maisonette                   825
attic-flat                          718
penthouse                           685
terrace-house                       556
chalet                              453
stepped-apartment                   236
farmhouse                           184
rustico                             111
loft                                 69
studio                               65
stepped-house                        50
furnished-residential-property       45
castle                               13
attic-room                           11
single-room                           4
secondary-suite                       3
detached-secondary-suite              3
Name: count, dtype: int64

### Andere Spalten
Es gibt noch viele andere Spalten, die äquivalente Informationen tragen, wie 'location', 'location_parsed', 'address' etc. Aber diese Spalten werden wir für das Projekt nicht verwenden, da wir kein Natural Language Processing integrieren werden.

## Spalten zusammenführen

Nachdem wir die Haupt-Spalten bestummen haben, können wir nun die Spalten mergen. Schauen wir aber zuerst noch die Anzahl NA-Werte der zu mergenden Spalten an, damit wir wissen, wie viele NA-Werte wir nach dem Mergen haben werden.

In [263]:
print(df['Living_area_unified'].isna().sum())
print(df['Municipality'].isna().sum())
print(df['Availability_merged'].isna().sum())
print(df['Floor_unified'].isna().sum())
print(df['Floor_space_merged'].isna().sum())
print(df['Plot_area_unified'].isna().sum())
print(df['Rooms_new'].isna().sum())

2125
736
12427
14187
22197
15280
11096


In [264]:
def merge_columns(row, main_column, columns_to_merge):
    if pd.isna(row[main_column]):
        for col in columns_to_merge:
            if pd.notna(row[col]):
                row[main_column] = row[col]
                break
    return row

df = df.apply(lambda row: merge_columns(row, 'Living_area_unified', ['Living space', 'detail_responsive#surface_living', 'Wohnfläche', 'Surface habitable', 'Superficie abitabile', 'Living_space_merged', 'Surface living:', 'Space extracted', 'space_cleaned']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Municipality', ['detail_responsive#municipality', 'Gemeinde', 'Commune', 'Comune', 'Municipality_merged']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Availability_merged', ['Availability', 'detail_responsive#available_from', 'Verfügbarkeit', 'Disponibilité', 'Disponibilità']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Floor_unified', ['detail_responsive#floor', 'Étage', 'Piano','Stockwerk', 'Floor_merged', 'Floor']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Floor_space_merged', ['Floor space', 'Nutzfläche', 'detail_responsive#surface_usable', 'Surface utile', 'Superficie utile']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Plot_area_unified', ['Plot area', 'detail_responsive#surface_property', 'Grundstücksfläche', 'Surface du terrain', 'Superficie del terreno', 'Plot_area_merged', 'Land area:']), axis=1)
df = df.apply(lambda row: merge_columns(row, 'Rooms_new', ['rooms', 'No. of rooms:']), axis=1)

Überprüfen wir die Zusammenführungen der Spalten.

In [265]:
print(df['Living_area_unified'].isna().sum())
print(df['Municipality'].isna().sum())
print(df['Availability_merged'].isna().sum())
print(df['Floor_unified'].isna().sum())
print(df['Floor_space_merged'].isna().sum())
print(df['Plot_area_unified'].isna().sum())
print(df['Rooms_new'].isna().sum())

1715
181
12094
13937
22131
15072
909


Die Ergebnisse zeigen, dass wir durch das Merging von Spalten eine erhebliche Menge an Daten "retten" konnten, die sonst verloren gegangen wären. Dies verbessert die Datendichte und die mögliche Aussagekraft unserer nachfolgenden Analysen erheblich.

## Kantone hinzufügen

Hier werden wir die Kantone zu den PLZ hinzufügen. Wir verwenden dazu die offizielle Liste der Postleitzahlen der Schweiz.

Quelle: [Swisspost Opendatasoft](https://swisspost.opendatasoft.com/api/explore/v2.1/catalog/datasets/plz_verzeichnis_v2/exports/csv)

In [266]:
df_plz = pd.read_csv('../../data/plz_verzeichnis_v2.csv', sep=';')
df_plz = df_plz[['GPLZ', 'KANTON']]
df_plz = df_plz.rename(columns={'GPLZ': 'gplz', 'KANTON': 'kanton'})

# Remove duplicates
df_plz = df_plz.drop_duplicates(subset=['gplz'], keep='first')

# Merge with main dataframe
df = df.merge(df_plz, left_on='Zip', right_on='gplz', how='left')

## Spalten Vereinheitlichen

Hier werden wir die Werte in den Spalten vereinheitlichen, z.B. Einheiten oder Vorzeichen entfernen und in numerische / kategorische Werte umwandeln.

In [267]:
def clean_area_values(column):
    column = column.astype(str)
    column = column.str.replace(' m²', '').str.replace(' m2', '')
    column = column.str.replace(',', '')
    column = pd.to_numeric(column, errors='coerce')
    return column

def clean_floor_values(column):
    column = column.astype(str)
    column = column.str.split('.', expand=True)[0]
    column = column.str.replace('.', '')
    column = column.str.replace('Ground floor', '0', case=False)
    column = column.str.replace('GF', '0', case=False)
    column = pd.to_numeric(column, errors='coerce')
    return column

def clean_room_values(column):
    column = column.astype(str)
    column = column.str.replace('rm', '', case=False)
    column = pd.to_numeric(column, errors='coerce')
    return column
    
def clean_type(x):
    """
    Returns "Other" if the property type has less than 30 entries, except for 'castle'
    """
    if x == 'castle':
        return x
    
    if x in df['type_unified'].value_counts()[df['type_unified'].value_counts() > 30].index:
        return x
    else:
        return "Other" 

def clean_zip(x):
    """
    Returns the first 2 digits of the zip code to unify the zip codes (example: 5000 and 5001 both become 50)
    """
    return str(x)[:2]
    
def to_date(x):
    """
    Convert the date strings to datetime format for easy comparison
    """
    try:
        return pd.to_datetime(x, format='%d.%m.%Y')
    except:
        return x
    
def categorize_availability(x):
    """
    Categorize the dates into broader time periods
    """
    today = pd.Timestamp.today()
    
    if x == "On request":
        return "On request"
    elif x == "Immediately":
        return "Immediately"
    elif (x - today).days <= 30:
        return "Within 1 month"
    elif (x - today).days <= 90:
        return "Within 3 months"
    elif (x - today).days <= 180:
        return "Within 6 months"
    else:
        return "More than 6 months"
    
# Apply the cleaning functions
df['Living_area_unified'] = clean_area_values(df['Living_area_unified'])
df['Floor_unified'] = clean_floor_values(df['Floor_unified'])
df['Floor_space_merged'] = clean_area_values(df['Floor_space_merged'])
df['Plot_area_unified'] = clean_area_values(df['Plot_area_unified'])
df['Rooms_new'] = clean_room_values(df['Rooms_new'])
df['type_unified'] = df['type_unified'].apply(clean_type)
df['Zip'] = df['Zip'].apply(clean_zip)
df['Availability_merged'] = df['Availability_merged'].apply(to_date)
df['Availability_Categorized'] = df['Availability_merged'].apply(categorize_availability)

df = df.copy().drop(columns=['Availability_merged'])

/var/folders/l1/7lm0cqd933xgjvldmnsjpphh0000gn/T/ipykernel_54183/1141843429.py:78: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Availability_Categorized'] = df['Availability_merged'].apply(categorize_availability)


## Variablenselektion

Basierend auf unserer Datensäuberung und Erkenntnisse werden wir folgende Variablen für die Modellierungen berücksichtigen:

In [268]:
columns_to_keep = [
	"kanton",
	"Zip",
	"Year built:",
	"type_unified",
	"Availability_Categorized",
	"Living_area_unified",
	"Floor_unified", 
	"Floor_space_merged",
	"Plot_area_unified", 
	"Rooms_new",

    "distanceToTrainStation",

    "gde_workers_sector1", "gde_workers_sector2", "gde_workers_sector3", "gde_workers_total", 
    "gde_area_agriculture_percentage", 
    "gde_area_forest_percentage", 
    "gde_area_nonproductive_percentage", 
    "gde_area_settlement_percentage", 
    "gde_average_house_hold", 
    "gde_empty_apartments", 
    "gde_foreigners_percentage", 
    "gde_new_homes_per_1000", 
    "gde_pop_per_km2", 
    "gde_population", 
    "gde_private_apartments", 
    "gde_social_help_quota", 
    "gde_tax", 

    "gde_politics_bdp",
    "gde_politics_cvp",
    "gde_politics_evp",
    "gde_politics_fdp",
    "gde_politics_glp",
    "gde_politics_gps",
    "gde_politics_pda",
    "gde_politics_rights",
    "gde_politics_sp",
    "gde_politics_svp",

    "NoisePollutionRailwayL", "NoisePollutionRailwayM", "NoisePollutionRailwayS", 
    "NoisePollutionRoadL", "NoisePollutionRoadM", "NoisePollutionRoadS", 
    "PopulationDensityL", "PopulationDensityM", "PopulationDensityS", 
    "RiversAndLakesL", "RiversAndLakesM", "RiversAndLakesS", 
    "WorkplaceDensityL", "WorkplaceDensityM", "WorkplaceDensityS", 
    "ForestDensityL", "ForestDensityM", "ForestDensityS"
]

df = df[columns_to_keep]

print(df.shape)

(24556, 56)


## Fehlende Werte

In [269]:
# Check for missing values
missing_data = df.isna().sum().sort_values(ascending=False)
missing_percentage = (df.isnull().sum() / df.shape[0]).sort_values(ascending=False) * 100

# Combine the results into a single DataFrame
missing_df = pd.concat([missing_data, missing_percentage], axis=1, keys=['Total Missing', 'Percentage (%)'])
missing_df.head(10)

,Total Missing,Percentage (%)
Floor_space_merged,22131,90.124613
Year built:,15291,62.269914
Plot_area_unified,15072,61.378075
Floor_unified,13937,56.755986
gde_politics_pda,12031,48.994136
gde_politics_bdp,8751,35.636912
gde_politics_evp,7469,30.416192
gde_politics_rights,4667,19.005538
gde_politics_glp,3386,13.788891
Living_area_unified,1715,6.984036


Oben sehen wir die Spalten mit den meisten NA nachdem wir die Daten gesäubert und zusammengeführt haben. Die Liste beinhaltet viele Gemeinde-Spalten. Wir betrachten bei `Floor_space_merged` eine Anzahl von 22'131 (90%) fehlenden Werten.

Wir entfernen alle Variablen welche mehr als 60% fehlende Werte haben.

In [270]:
# Define a threshold for missing values (e.g., 80%)
threshold = 80

# Get the columns that have less than 80% missing values
columns_to_keep = missing_df[missing_df['Percentage (%)'] < threshold].index

# Keep only the columns that have less than 80% missing values
df = df[columns_to_keep]

# Check the shape of the DataFrame
print(df.shape)

(24556, 55)


Wichtig hier ist, dass wir die gleiche Spalten haben wie unser Trainings-Datensatz und das ist der Fall. Wir haben aber 1 Spalten weniger, da wir die Spalten `price_cleaned` nicht im Test-Datensatz haben.

## Daten speichern

Da wir die Daten nun bereinigt haben, speichern wir sie in einer neuen csv-Datei. Diese Datei werden wir in den nächsten Notebooks verwenden für die explorative Datenanalyse und die Modellierung.

In [271]:
df.to_csv('../../data/test_data_kaggle_clean.csv', index=False)